In [ ]:
import pandas as pd
from game.poker_oracle import PokerOracle
import utils.config as config
import numpy as np
from itertools import combinations

In [ ]:
oracle = PokerOracle()

In [ ]:
pub_cards = ("9♣", "9♦", "T♣")

In [ ]:
matrix = PokerOracle.calculate_utility_matrix(pub_cards)

In [ ]:
matrix[0, -1]

In [ ]:
matrix.tofile("matrix.csv", sep=",")

In [ ]:
matrix_width, matrix_height = matrix.shape

for i in range(matrix_width):
    for j in range(matrix_height):
        if not matrix[i, j] == -matrix[j, i]:
            print(f"{matrix[i, j]} - {matrix[j, i]}")

In [ ]:
print(np.all(matrix == -matrix.T), np.sum(matrix) == 0)

In [ ]:
matrix.shape

In [ ]:
l = matrix.tofile("matrix.csv", sep=",", format="%10.5f")

In [ ]:
PokerOracle.quantify_hand(["9♣", "T♥"], ["K♣", "A♦", "A♣"], oracle)

In [ ]:
deck = PokerOracle.generate_deck()
deck

In [ ]:
all_hole_cards = list(combinations(deck, 2)) == list(PokerOracle.all_hole_combinations())
all_hole_cards

In [ ]:
all_hole_cards = list(PokerOracle.all_hole_combinations())

In [ ]:
range = np.ones(len(all_hole_cards))
print(len(range), range)

In [ ]:
def cards_to_range_index(card1, card2):
    return all_hole_cards.index((card1, card2))

def range_index_to_cards(index):
    return all_hole_cards[index]

def cards_to_range_index_both_ways(card1, card2):
    return cards_to_range_index(card1, card2), cards_to_range_index(card2, card1)

In [ ]:
card1 = '9♠'
for card2 in deck:
    if card1 == card2:
        continue 
    index  = cards_to_range_index(card1, card2)
    print(f'{card1} {card2} {index} -- {range_index_to_cards(index)}')

In [ ]:
all_hole_cards

# Data gen

In [ ]:
import data.datagen as datagen
from state.state_manager import PokerGameStage
import multiprocessing as mp

In [ ]:
def generate_data(start, end, stage, n_pub):
    local_data = []
    for i in range(start, end):
        if i % 10 == 0:
            print(i)
        local_data.append(datagen.get_random_example((stage, n_pub)))
    return local_data

In [ ]:
def get_data_mp(stage, n_pub):
    num_processes = 8
    total_items = 1000

    pool = mp.Pool(processes=num_processes)

    # Calculate the chunk size for each process
    chunk_size = total_items // num_processes
    ranges = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_processes)]
    ranges[-1] = (ranges[-1][0], total_items)  # Ensure the last range covers the remainder

    # Map the generate_data function to the data ranges
    results = [pool.apply_async(generate_data, args=(r[0], r[1], stage, n_pub)) for r in ranges]

    # Close the pool and wait for the work to finish
    pool.close()
    pool.join()

    # Collect all results from the processes
    data = []
    for result in results:
        data.extend(result.get())

    return data

In [ ]:
n_pub = 4
stage = PokerGameStage.TURN
new_data = get_data_mp(stage, n_pub)

In [ ]:
data = []
for i in range(1000):
    if i % 10 == 0:
        print(i)
    data.append(datagen.get_random_example((stage, n_pub)))

In [ ]:
import pickle as pkl
import torch
test_frac = 0.2
size = 1000
test_num = int(size * test_frac)

In [ ]:
pkl.dump(data, open("data.pkl", "wb"))

In [ ]:
dataset = torch.cat(data)

In [ ]:
train_dataset, test_dataset = dataset.split([size - test_num, test_num])

In [ ]:
print(train_dataset.shape, test_dataset.shape)

In [ ]:
torch.save(train_dataset, "train_dataset.pt")
torch.save(test_dataset, "test_dataset.pt")

# Model training

In [ ]:
import torch
d = torch.load("data/RIVER/train1.pt")
import pandas as pd
import numpy as np
from game.poker_oracle import PokerOracle

In [ ]:
pot_index = 276*2+5

In [ ]:
invecs, cards, pot, p1, p2, _ = d.split([
    276 * 2,
    5,
    1,
    276,
    276,
    1], dim=1)

In [ ]:
torch.mean(d[:, pot_index])

In [ ]:
def public_cards_to_index(cards_col):
    
    idx = []
    deck = PokerOracle.generate_deck()
    for cards in cards_col:
        cards = cards.replace('clubs', '♣')
        cards = cards.replace('hearts', '♥')
        cards = cards.replace('diamonds', '♦')
        cards = cards.replace('spades', '♠')
        cards = cards.replace('10', 'T')
        idx.append(deck.index(cards))

    return idx

In [ ]:
public_cards_to_index(df['public_cards'][0])

In [ ]:
df['public_cards'] = df['public_cards'].apply(public_cards_to_index)

In [ ]:
df

In [ ]:
df = pd.read_csv("turn.csv")
def parse_vector(vector_str):
    return np.array(list(map(float, vector_str.split(','))))

def parse_array(s):
    return np.array([float(item) for item in s.strip('[]').split(',')])

def parse_cards(s):
    return s.strip('[]').replace("'", "").split(', ')

def public_cards_to_index(cards_col):
    idx = []
    deck = PokerOracle.generate_deck()
    for cards in cards_col:
        cards = cards.replace('clubs', '♣')
        cards = cards.replace('hearts', '♥')
        cards = cards.replace('diamonds', '♦')
        cards = cards.replace('spades', '♠')
        cards = cards.replace('10', 'T')
        idx.append(deck.index(cards))

    return idx


df['r1'] = df['r1'].apply(parse_array)
df['r2'] = df['r2'].apply(parse_array)
df['v1'] = df['v1'].apply(parse_array)
df['v2'] = df['v2'].apply(parse_array)
df['public_cards'] = df['public_cards'].apply(parse_cards)
df['public_cards'] = df['public_cards'].apply(public_cards_to_index)

In [ ]:
print(df['r1'].tolist()[0])  # Print first element to check its structure


In [ ]:
def df_to_tensor(df):
    tensors = []
    
    # Add `r1` and `r2` first
    tensors.append(torch.tensor(df['r1'].tolist(), dtype=torch.float))
    tensors.append(torch.tensor(df['r2'].tolist(), dtype=torch.float))

    # Add `public_cards` indices as tensors (convert to long if using as indices)
    tensors.append(torch.tensor(df['public_cards'].tolist(), dtype=torch.long))

    # Add `pot`
    tensors.append(torch.tensor(df['pot'].values, dtype=torch.float).view(-1, 1))

    # Add `v1` and `v2`
    tensors.append(torch.tensor(df['v1'].tolist(), dtype=torch.float))
    tensors.append(torch.tensor(df['v2'].tolist(), dtype=torch.float))

    # Concatenate all tensors along columns
    full_tensor = torch.cat(tensors, dim=1)
    return full_tensor

# Convert the DataFrame to a tensor with the correct order
data_tensor = df_to_tensor(df)


In [ ]:
data_tensor.shape

In [ ]:
torch.save(data_tensor, "data_tensor.pt")

In [ ]:
d.shape

In [ ]:
# get sum and std dev of pot
pot_sum = torch.mean(pot)
pot_std = torch.std(pot)
print(pot_sum, pot_std)

In [ ]:
from nn.nn_training import NNTrainer
from state.state_manager import PokerGameStage
stage = PokerGameStage.FLOP

In [ ]:
trainer = NNTrainer()

In [ ]:
trainer.train_network(stage, 150, 5000)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv("lightning_logs/FLOP/lightning_logs/version_0/metrics.csv")
df_train_loss = df[df["train_loss"].notnull()]
df_val_loss = df[df["val_loss"].notnull()]
df_accum_train_loss = df_train_loss.groupby('epoch').mean()
df_accum_val_loss = df_val_loss.groupby('epoch').mean()

In [ ]:
# Val loss
plt.plot(df_accum_val_loss['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['val'], loc='upper left')
plt.show()

In [ ]:
# Val loss
plt.plot(df_accum_train_loss['train_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()